# Intermediate Julia for scientific computing

This workshop is designed to introduce two fundamental concepts in Julia: **types** and **metaprogramming**.

In order to cover various key uses of types in Julia, we have chosen to frame the discussion around a concrete topic in scientific computing, namely **root-finding**. 
The goal is *not* to learn algorithms for root finding *per se*, but rather to have a (pseudo-)real context in which to explore various concepts centered around types and how they arise naturally in real applications of Julia, in particular applications of **multiple dispatch**, which is one of the core choices in Julia that differentiate it from other common languages.

We will implement a couple of root-finding algorithms just to have something to work with. These will just be toy implementations that are far away from the best implementations. 

Instead we should use one of the high-quality packages that are available in Julia for this purpose. The large number of them shows the importance of root finding. The ones that I am aware of are the following (in alphabetical order):

- Single root of a nonlinear function:
    - [`NLsolve.jl`](https://github.com/JuliaNLSolvers/NLsolve.jl)
    - [`Roots.jl`](https://github.com/JuliaMath/Roots.jl)

- All roots of polynomial:
    - [`HomotopyContinuation.jl`](https://www.juliahomotopycontinuation.org)
    - [`PolynomialRoots.jl`](https://github.com/giordano/PolynomialRoots.jl)
    - [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl)
    
- All roots of a nonlinear function:
    - [`ApproxFun.jl`](https://github.com/JuliaApproximation/ApproxFun.jl)
    - [`IntervalRootFinding.jl`](https://github.com/JuliaIntervals/IntervalRootFinding.jl)
    - [`MDBM.jl`](https://github.com/bachrathyd/MDBM.jl)
    - [`Roots.jl`](https://github.com/JuliaMath/Roots.jl)

Each of these uses different techniques, with different advantages and disadvantages.

The challenge exercise for the workshop is: develop a package which integrates all of these disparate packages into a coherent whole!

### Logistics of the workshop

The workshop is based around a series of exercises to be done during the workshop. We will pause to work on the exercises and then I will discuss possible solutions during the workshop.

These techniques are useful for both users and developers; indeed, in Julia the distinction between users and developers is not useful, since it's much easier than in other languages to join the two categories together.

### Outline

We will start by quickly reviewing roots of functions and quickly reviewing one of the standard algorithms, **Newton's algorithm**. We will restrict to finding roots of 1D functions for simplicity.

Newton's algorithm requires the calculation of derivatives, for which several choices of algorithm are available. We will see how to encode the choice of algorithm using dispatch.

Then we will define types which will contain all information about a root-finding problem.

## Roots

Given a function $f: \mathbb{R} \to \mathbb{R}$ (i.e. that accepts a single real number as argument and returns another real number), recall that a **root** or **zero** of the function is a number $x^*$ such that

$$ f(x^*) = 0, $$

i.e. it is a solution of the equation $f(x) = 0$.

In general it is impossible to solve this equation exactly for $x^*$, so we use iterative numerical algorithms instead.

#### Example

Recall that the function $f$ given by $f(x) := x^2 - 2$ has exactly two roots, at $x^*_1 = +\sqrt{2}$ and $x^*_2 = -\sqrt{2}$. Note that it is impossible to represent these values exactly using floating-point arithmetic.

## Newton algorithm

The Newton algorithm for (possibly) finding a root of a nonlinear function $f(x)$ in 1D is the following iteration:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)},$$

where $f'$ is the derivative of $f$. We start from an initial guess $x_0$ that can be almost anything (except points for which $f'(x_0) = 0$).

#### Exercise 1

1. Implement the Newton algorithm for a fixed number $n$ of steps in a function `newton`, starting from a given starting point $x_0$.  

    Hint: Which information does the function require?


2. Does your function work with other number types, such as `BigFloat`? What do you need in order to run it with those types? Use it to calculate $\sqrt{2}$. How many decimal places are correct with the standard precision of `BigFloat`?

## Calculating derivatives

The Newton algorithm requires us to specify the derivative of a function. If $f$ is a complicated function, we certainly don't want to do that by hand.

One standard solution is to use a *finite-difference approximation*:

$$f'(a) \simeq \frac{f(a + h) - f(a - h)}{2h}.$$

#### Exercise 2

1. Implement a function `finite_difference` with a default value $h = 0.001$.


2. Implement a version of `newton` that does not take the derivative as argument and uses `finite_difference` to calculate the derivative. This version of `newton` should **re-use** the previous version by defining the function `fp` and calling that version.

### Algorithmic differentiation

An alternative way to calculate derivatives is by using [**algorithmic differentiation**](https://en.wikipedia.org/wiki/Automatic_differentiation) (also called **automatic differentiation** or **computational differentiation**). This gives exact results (up to rounding error).


We will implement this algorithm in the next notebook, but for now let's just use the implementation in the excellent [`ForwardDiff.jl` package](https://github.com/JuliaDiff/ForwardDiff.jl).


#### Exercise 3

1. Install `ForwardDiff.jl` if necessary.


2. Import it.


3. Define a function `forwarddiff` that uses the `ForwardDiff.derivative` function to calculate a derivative.

### Choosing between algorithms

We now have two different algorithms available to calculate derivatives. This kind of situation is common in scientific computing; for example, the [`DifferentialEquations.jl`](http://docs.juliadiffeq.org/latest/) ecosystem has some 300 algorithms for solving differential equations. One of the techniques we will learn is how to easily be able to specify different algorithms.

One possible solution is just by specifying the *function* to use as an argument to another function:

#### Exercise 4

1. Make a version of the Newton algorithm that takes an argument which is the algorithm to use to calculate the derivative. 

    Hint: Here we will just pass in the function as an argument by giving its name as a parameter.

For some purposes this technique of passing functions may be sufficient. In a later notebook we will see a more powerful technique to specify different algorithms.